In [2]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

In [7]:
class preprocessing:
    def __init__(self,filepath,savepath):
        self.filepath=filepath
        self.savepath=savepath
    def preprocessing_property_beds(self):
        dataset=pd.read_csv(self.filepath)
        property_beds=pd.DataFrame(dataset['property_beds'])
        property_beds.columns=['property_beds']
        return property_beds        
    def preprocessing_property_bed_type(self):
        dataset=pd.read_csv(self.filepath)
        BedType=pd.get_dummies(dataset['property_bed_type'])
        property_bed_type=pd.DataFrame(BedType.values,columns=list(BedType.columns))
        return property_bed_type
    def preprocessing_property_amenities(self,amenities_list_path):  
        dataset=pd.read_csv(self.filepath)
        amenitieslist=pd.read_table(amenities_list_path,header=None,delim_whitespace=True)
        amenities=[]
        for amenities_element in amenitieslist[0]:
            amenities_element=amenities_element.replace(',','')
            amenities_element=amenities_element.replace("'","")
            amenities_element=amenities_element.replace(',','')
            amenities.append(amenities_element)
        PropertyAmenities=np.zeros([dataset.shape[0],len(amenities)])
        a = (map(lambda x: str(x).lower(), dataset['property_amenities']))#全小写
        dataset['property_amenities']=list(a)
        a =(map(lambda x: str(x).replace(' ',''),dataset['property_amenities']))#去空格
        dataset['property_amenities']=list(a)
        i=0
        for pal in amenities:
            for n in range(dataset.shape[0]):
                tempj=(lambda x:pal in str(x).split(','))(dataset['property_amenities'][n])
                if tempj==True:
                    PropertyAmenities[n,i]=1
                else:
                    PropertyAmenities[n,i]=0
            i+=1
        property_amenities=pd.DataFrame(PropertyAmenities,columns=amenities)
        return property_amenities
    def preprocessing_property_scraped_at(self):
        dataset=pd.read_csv(self.filepath)
        ScrapedAt=pd.to_datetime(dataset['property_scraped_at'])
        property_scraped_at=pd.DataFrame(ScrapedAt)
        property_scraped_at.columns=['property_scraped_at']
        
        reviews_last=pd.to_datetime(dataset['reviews_last'])
        Scraped_Minus_Review=ScrapedAt-reviews_last
        scraped_minus_review=pd.DataFrame(Scraped_Minus_Review,columns=['scraped_minus_review'])
        return property_scraped_at,scraped_minus_review
    
    def preprocessing_property_last_updated(self):
        dataset=pd.read_csv(self.filepath)
        last_updated=np.zeros([dataset.shape[0],])
        n=0
        for plu in dataset['property_last_updated']:
        #print(plu)
            if plu=='today':
                last_updated[n]=0
            elif plu=='yesterday':
                last_updated[n]=1
            elif list(filter(lambda x: 'day' in str(x),plu.split(' ')))!=[]:
                last_updated[n]=1*int(plu.split(' ')[0])
            elif list(filter(lambda x: 'week' in str(x),plu.split(' ')))!=[]:
                if plu.split(' ')[0]=='a':
                    last_updated[n]=7
                else:
                    last_updated[n]=7*int(plu.split(' ')[0])
            elif list(filter(lambda x: 'month' in str(x),plu.split(' ')))!=[]:
                if plu.split(' ')[0]=='a':
                    last_updated[n]=30
                else:
                    last_updated[n]=30*int(plu.split(' ')[0])
            n+=1
        property_last_updated=pd.DataFrame(last_updated,columns=['property_last_updated'])
        return property_last_updated
    def preprocessing_host_since(self):
        dataset=pd.read_csv(self.filepath)
        host_since=pd.DataFrame(pd.to_datetime(dataset['host_since']))
        return host_since
    def preprocessing_host_location(self):
        dataset=pd.read_csv(self.filepath)
        a = (map(lambda x: str(x).lower(), dataset['host_location']))
        property_host_location=list(a)
        a =(map(lambda x: str(x).replace(' ',''),property_host_location))
        property_host_location=list(a)
        HostLocation=[]
        for hostlocation in property_host_location:
            hostlocation_temp=hostlocation.split(',')
            if len(hostlocation_temp)!=1:
                if hostlocation_temp[-1]=='belgium':
                    if hostlocation_temp[-2]=='brussels':
                        HostLocation.append(1)
                    elif hostlocation_temp[-2]=='flanders':
                        HostLocation.append(2)
                    else:
                        HostLocation.append(3)
                elif hostlocation_temp[-1]!='belgium':
                    HostLocation.append(4)
            elif len(hostlocation_temp)==1:
                if hostlocation_temp=='be':
                    HostLocation.append(3)
                else:
                    HostLocation.append(4)
            else:
                HostLocation.append(4)
        host_location=pd.DataFrame(HostLocation,columns=['host_location'])
        host_location=pd.get_dummies(host_location['host_location'])
        host_location.columns=['host_location_belgium_brussels','host_location_belgium_flanders','host_location_belgium_other_regions','host_location_other_countries']
        return host_location
    def data_concat(self,amenities_list_path):
        property_beds=self.preprocessing_property_beds()
        property_bed_type=self.preprocessing_property_bed_type()
        property_amenities=self.preprocessing_property_amenities(amenities_list_path)
        scraped_at,scraped_minus_review=self.preprocessing_property_scraped_at()
        property_last_updated=self.preprocessing_property_last_updated()
        host_since=self.preprocessing_host_since()
        host_location=self.preprocessing_host_location()
        dataset_after=pd.concat([property_beds,property_bed_type,property_amenities,scraped_at,scraped_minus_review,host_since,host_location],axis=1)
        return dataset_after
    

In [8]:
filepath='.../data/test.csv'
savepath='.../test/'
amenities_list_path='.../amenitieslist.txt'
pp=preprocessing(filepath,savepath)
dataset_after=pp.data_concat(amenities_list_path)

In [9]:
dataset_after.head(10)

,property_beds,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,24-hourcheck-in,accessible-heightbed,airconditioning,babybath,...,widehallwayclearance,windowguards,wirelessinternet,property_scraped_at,scraped_minus_review,host_since,host_location_belgium_brussels,host_location_belgium_flanders,host_location_belgium_other_regions,host_location_other_countries
0,2.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,1 days,2013-07-25,0,0,0,1
1,1.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-08,NaT,2016-06-12,1,0,0,0
2,3.0,0,0,0,0,1,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,37 days,2013-07-26,1,0,0,0
3,1.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,2 days,2016-06-12,1,0,0,0
4,1.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,NaT,2016-06-12,1,0,0,0
5,1.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,NaT,2016-06-12,1,0,0,0
6,1.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,NaT,2016-06-13,1,0,0,0
7,2.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,260 days,2016-06-13,0,1,0,0
8,2.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-09,NaT,2013-07-26,1,0,0,0
9,2.0,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2017-05-08,297 days,2016-06-13,1,0,0,0


In [ ]:
dataset_after.to_csv(savepath+'test_after_processing.csv')